In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [2]:
import sys
from pathlib import Path
print(Path.cwd())

/Users/vaishnukanna/Documents/jegadeesh-titman-momentum/results


In [3]:
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

In [4]:
from src.ff3_reg import ff3_reg
from src.config import DATA_DIR

In [5]:
path1 = DATA_DIR / "portfolio" / "J6_K6_skip0_returns.csv"
if not path1.exists():
        raise FileNotFoundError(f"Portfolio data not found at {path1}. Run portfolio.py first.")
else:
    portfolio = pd.read_csv(
        path1, 
        parse_dates=["Date"]
        ).set_index("Date")["spread"]
    print(portfolio.head())

Date
2001-02-01    0.212342
2001-03-01   -0.007522
2001-04-01   -0.139201
2001-05-01    0.048643
2001-06-01   -0.018337
Name: spread, dtype: float64


In [6]:
path2 = DATA_DIR / "portfolio" / "ff3.csv"
if not path2.exists():
    raise FileNotFoundError(f"Risk-free rate data not found at {path2}. Run download_ff3s.py first.")
else:
    ff3_factors = pd.read_csv(
        path2,
        parse_dates=["Date"]
        ).set_index("Date")

    print(ff3_factors.head())

                RM     SMB     HML      RF
Date                                      
1926-07-01  0.0289 -0.0255 -0.0239  0.0022
1926-08-01  0.0264 -0.0114  0.0381  0.0025
1926-09-01  0.0038 -0.0136  0.0005  0.0023
1926-10-01 -0.0327 -0.0014  0.0082  0.0032
1926-11-01  0.0254 -0.0011 -0.0061  0.0031


In [7]:
df = pd.concat(
    [portfolio, ff3_factors],
    axis=1
).dropna()

df.head()

,spread,RM,SMB,HML,RF
Date,,,,,
2001-02-01,0.212342,-0.1003,-0.0065,0.1223,0.0038
2001-03-01,-0.007522,-0.0725,0.0026,0.0647,0.0042
2001-04-01,-0.139201,0.0793,0.0050,-0.0464,0.0039
2001-05-01,0.048643,0.0068,0.0248,0.0338,0.0032
2001-06-01,-0.018337,-0.0194,0.0635,-0.0126,0.0028


In [8]:
ff3_res = ff3_reg(
    portfolio_returns=df['spread'],
    ff_factors=df[['RM', 'SMB', 'HML', 'RF']]
)

ff3_res

const    0.017015
RM      -0.509983
SMB      0.097264
HML     -0.136381
dtype: float64
Index(['const', 'RM', 'SMB', 'HML'], dtype='object')


{'alpha': np.float64(0.017014643154785878),
 't_alpha': np.float64(5.4028616063443495),
 'beta_mkt': np.float64(-0.5099832135056457),
 'beta_smb': np.float64(0.09726361111558517),
 'beta_hml': np.float64(-0.13638074463785219),
 't_beta_mkt': np.float64(-7.124302280775026),
 't_beta_smb': np.float64(0.7652053125995144),
 't_beta_hml': np.float64(-1.3953466986633993),
 'r_squared': np.float64(0.17121924083305806)}

In [9]:
pd.DataFrame([ff3_res]).to_csv(
    PROJECT_ROOT/ "results" / "ff3.csv",
    index=False
)

- The winner–loser momentum portfolio earns a monthly alpha of 1.7%, which is statistically significant with a t-statistic of 5.4; the portfolio exhibits a strongly negative and significant market beta, indicating that momentum returns are not compensation for systematic market risk. 
- Loadings on the size and value factors are small and statistically insignificant, suggesting that momentum profits are not driven by size or value effects. 
- Overall, the Fama–French three-factor model explains only a modest fraction of the variation in momentum returns, consistent with the interpretation that momentum represents an independent return anomaly.